# Chapter 7.1: Contextual Chunk Headers (CCH)

이 노트북은 OSTEP RAG 시스템에 Contextual Chunk Headers (CCH) 기법을 적용하여 검색 품질을 개선하는 방법을 실습합니다.

## 📚 학습 목표
- CCH 개념 이해: 청크에 문서 컨텍스트(챕터 정보) 추가의 중요성
- 기존 청크 vs CCH 적용 청크의 검색 성능 비교
- CCH 적용 FAISS 인덱스 생성 및 저장

## 📋 실습 구성
1) 설정 및 하이퍼파라미터 정의
2) 기존 데이터 로드 (청크/인덱스)
3) CCH 개념 설명 및 헤더 생성
4) 비교 실험: 일반 검색 vs CCH 검색
5) CCH 적용 인덱스 생성 및 저장
6) 데모: CCH 기반 검색 테스트

> **핵심 아이디어**: 청크에 상위 컨텍스트(챕터 제목 등)를 헤더로 추가하면, 임베딩이 더 풍부한 의미를 담게 되어 검색 정확도가 향상됩니다.


---
## 1️⃣ 설정 및 하이퍼파라미터 정의

이 셀에서는 경로, 모델, 검색 파라미터 등을 정의합니다.

**주요 내용:**
- 기존 산출물 경로 (청크 JSON, FAISS 인덱스)
- CCH 적용 후 저장할 새 인덱스 경로
- 임베딩 모델 및 검색 하이퍼파라미터

**실행 결과:**
- 설정값이 출력됩니다.


In [ ]:
# ========================================
# 1️⃣ 설정 / 하이퍼파라미터
# ========================================
import os
import json
import faiss
import numpy as np
import torch
from pathlib import Path
from typing import List, Dict, Any

# 재현성
SEED = 42
np.random.seed(SEED)

# 디바이스
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 경로 (기존 산출물)
CHUNK_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/chunk/ostep_tok400_ov20.json"
INDEX_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw.index"
INDEX_META_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_metadata.json"

# CCH 적용 후 저장 경로
CCH_INDEX_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch.index"
CCH_META_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch_metadata.json"

# 검색 하이퍼파라미터
TOP_K = 5
EF_SEARCH = 64

# 임베딩 모델
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
NORMALIZE = True

print(f"🔧 Device: {DEVICE}")
print(f"📁 원본 청크: {CHUNK_FILE}")
print(f"📁 원본 인덱스: {INDEX_FILE}")
print(f"📁 CCH 인덱스: {CCH_INDEX_FILE}")
print(f"🔎 TOP_K={TOP_K}, efSearch={EF_SEARCH}")
print(f"🧠 임베딩 모델: {EMBED_MODEL}")


---
## 2️⃣ 기존 데이터 로드

Chapter 2-4에서 생성한 청크 JSON과 FAISS 인덱스를 로드합니다.

**주요 내용:**
- 청크 데이터 로드 및 구조 확인
- 기존 FAISS 인덱스 로드
- 임베딩 모델 로드

**실행 결과:**
- 청크 개수와 샘플이 출력됩니다.


In [ ]:
# 청크 로드
print("청크 데이터 로드 중...")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    CHUNKS = json.load(f)
print(f"✓ 청크 로드 완료: {len(CHUNKS)}개")

# 샘플 출력
print("\n샘플 청크 구조:")
sample = CHUNKS[0]
for key, value in sample.items():
    if key == 'text':
        print(f"  {key}: {value[:100]}...")
    else:
        print(f"  {key}: {value}")

# 기존 인덱스 로드
print("\n기존 인덱스 로드 중...")
INDEX_ORIGINAL = faiss.read_index(INDEX_FILE)
print(f"✓ 인덱스 로드 완료: {INDEX_ORIGINAL.ntotal}개 벡터")

# 임베딩 모델 로드
print(f"\n임베딩 모델 로드 중: {EMBED_MODEL}")
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(EMBED_MODEL, device="cpu" if DEVICE=="cpu" else DEVICE)
print("✓ 임베딩 모델 로드 완료")


---
## 3️⃣ CCH 개념 및 헤더 생성

이 셀에서는 CCH의 핵심 개념을 설명하고 실제 헤더를 생성합니다.

**CCH란?**
- 개별 청크는 종종 대명사나 암묵적 참조를 포함하여 문맥이 부족합니다
- 문제점: "이 알고리즘은..." → 어떤 알고리즘인지 청크만으로는 알 수 없음
- 해결: 청크 앞에 "Document: [챕터 제목]" 같은 헤더를 추가
- 효과: 임베딩이 더 완전한 컨텍스트를 담게 되어 검색 정확도 향상

**주요 내용:**
- 헤더 생성 함수 정의
- 샘플 청크의 헤더 포함/미포함 버전 비교

**실행 결과:**
- 헤더가 추가된 청크 예시가 출력됩니다.


In [ ]:
def create_chunk_header(chunk: Dict[str, Any]) -> str:
    """
    청크에 대한 컨텍스트 헤더를 생성합니다.
    
    Args:
        chunk: 청크 딕셔너리 (chapter_title, subsection_title 등 포함)
    
    Returns:
        헤더 문자열
    """
    chapter_title = chunk.get('chapter_title', 'Unknown Chapter')
    subsection_title = chunk.get('subsection_title')
    
    header = f"Document: {chapter_title}"
    if subsection_title:
        header += f" - {subsection_title}"
    
    return header

def add_header_to_chunk(chunk: Dict[str, Any]) -> str:
    """
    청크 텍스트에 헤더를 추가합니다.
    
    Args:
        chunk: 청크 딕셔너리
    
    Returns:
        헤더 + 원본 텍스트
    """
    header = create_chunk_header(chunk)
    text = chunk.get('text', '')
    return f"{header}\n\n{text}"

# 샘플 비교
print("=" * 80)
print("[샘플 청크 비교: 헤더 없음 vs 헤더 있음]")
print("=" * 80)

sample_idx = 10
sample_chunk = CHUNKS[sample_idx]

print("\n[원본 청크 (헤더 없음)]:")
print(sample_chunk['text'][:300] + "...")

print("\n[CCH 적용 청크 (헤더 있음)]:")
cch_text = add_header_to_chunk(sample_chunk)
print(cch_text[:300] + "...")

print("\n💡 차이점: 헤더가 청크에 명시적인 문서 컨텍스트를 제공합니다.")


---
## 4️⃣ 비교 실험: 일반 검색 vs CCH 검색

이 셀에서는 동일한 쿼리에 대해 일반 검색과 CCH 검색을 비교합니다.

**주요 내용:**
- 테스트 쿼리 정의
- 기존 인덱스로 검색 (헤더 없음)
- CCH 텍스트로 임시 임베딩 생성 및 검색
- 결과 비교

**실행 결과:**
- 두 방식의 검색 결과와 유사도 점수가 출력됩니다.


In [ ]:
def distance_to_similarity(distances: np.ndarray) -> np.ndarray:
    """L2 거리를 유사도로 변환 (정규화된 벡터 가정)"""
    return 1.0 - (distances / 2.0)

def search_with_index(query: str, index: faiss.Index, k: int = TOP_K):
    """FAISS 인덱스로 검색"""
    faiss.ParameterSpace().set_index_parameter(index, "efSearch", EF_SEARCH)
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=NORMALIZE).astype('float32')
    D, I = index.search(q_emb, k)
    scores = distance_to_similarity(D[0])
    return I[0], scores

# 테스트 쿼리
TEST_QUERY = "How does the operating system manage virtual memory?"

print("=" * 80)
print(f"테스트 쿼리: {TEST_QUERY}")
print("=" * 80)

# 1) 일반 검색 (헤더 없음)
print("\n[일반 검색 결과 (헤더 없음)]:")
indices_orig, scores_orig = search_with_index(TEST_QUERY, INDEX_ORIGINAL, k=3)
for rank, (idx, score) in enumerate(zip(indices_orig, scores_orig), 1):
    if idx >= 0:
        chunk = CHUNKS[idx]
        print(f"\n{rank}. Score: {score:.4f}")
        print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
        print(f"   Preview: {chunk['text'][:150]}...")

print("\n" + "=" * 80)
print("💡 CCH 비교를 위해서는 CCH 인덱스를 먼저 생성해야 합니다.")
print("   다음 단계에서 CCH 인덱스를 생성합니다.")
print("=" * 80)


---
## 5️⃣ CCH 적용 인덱스 생성 및 저장

이 셀에서는 모든 청크에 CCH를 적용하여 새로운 FAISS 인덱스를 생성합니다.

**주요 내용:**
- 모든 청크에 헤더 추가
- CCH 적용 텍스트 임베딩 생성
- HNSW 인덱스 생성 및 저장

**실행 결과:**
- CCH 인덱스 생성 진행상황과 저장 경로가 출력됩니다.
- 이 과정은 몇 분 소요될 수 있습니다.


In [ ]:
print("CCH 적용 임베딩 생성 중...")
print(f"총 {len(CHUNKS)}개 청크 처리")

# 모든 청크에 헤더 추가
cch_texts = [add_header_to_chunk(chunk) for chunk in CHUNKS]
print(f"✓ 헤더 추가 완료: {len(cch_texts)}개")

# 임베딩 생성 (배치 처리)
BATCH_SIZE = 128
print(f"\n임베딩 생성 중 (배치 크기: {BATCH_SIZE})...")
all_embeddings = []
for i in range(0, len(cch_texts), BATCH_SIZE):
    batch = cch_texts[i:i+BATCH_SIZE]
    batch_emb = embed_model.encode(
        batch,
        convert_to_numpy=True,
        normalize_embeddings=NORMALIZE,
        show_progress_bar=False
    )
    all_embeddings.append(batch_emb)
    if (i // BATCH_SIZE + 1) % 10 == 0:
        print(f"  진행: {i+len(batch)}/{len(cch_texts)} 청크")

embeddings = np.vstack(all_embeddings).astype('float32')
print(f"✓ 임베딩 생성 완료: shape={embeddings.shape}")

# HNSW 인덱스 생성
print("\nHNSW 인덱스 생성 중...")
DIM = embeddings.shape[1]
M = 32  # HNSW 파라미터
EF_CONSTRUCTION = 64

index_cch = faiss.IndexHNSWFlat(DIM, M)
index_cch.hnsw.efConstruction = EF_CONSTRUCTION
index_cch.add(embeddings)
print(f"✓ 인덱스 생성 완료: {index_cch.ntotal}개 벡터")

# 저장
print(f"\n인덱스 저장 중: {CCH_INDEX_FILE}")
os.makedirs(os.path.dirname(CCH_INDEX_FILE), exist_ok=True)
faiss.write_index(index_cch, CCH_INDEX_FILE)

# 메타데이터 저장
cch_metadata = {
    "index_type": "HNSW",
    "dimension": int(DIM),
    "num_vectors": int(index_cch.ntotal),
    "M": M,
    "efConstruction": EF_CONSTRUCTION,
    "model": EMBED_MODEL,
    "normalized": NORMALIZE,
    "cch_applied": True,
    "chunk_file": CHUNK_FILE
}
with open(CCH_META_FILE, 'w', encoding='utf-8') as f:
    json.dump(cch_metadata, f, ensure_ascii=False, indent=2)

print(f"✓ 메타데이터 저장 완료: {CCH_META_FILE}")
print("\n🎉 CCH 인덱스 생성 완료!")


---
## 6️⃣ 데모: CCH 기반 검색 테스트

이 셀에서는 CCH 인덱스를 사용하여 검색을 수행하고 결과를 비교합니다.

**주요 내용:**
- CCH 인덱스로 동일한 쿼리 검색
- 일반 검색 결과와 비교
- 검색 품질 개선 확인

**실행 결과:**
- CCH 검색 결과와 원본 검색 결과의 비교가 출력됩니다.


In [ ]:
# CCH 인덱스 검색
print("=" * 80)
print(f"테스트 쿼리: {TEST_QUERY}")
print("=" * 80)

print("\n[CCH 검색 결과 (헤더 포함)]:")
indices_cch, scores_cch = search_with_index(TEST_QUERY, index_cch, k=3)
for rank, (idx, score) in enumerate(zip(indices_cch, scores_cch), 1):
    if idx >= 0:
        chunk = CHUNKS[idx]
        print(f"\n{rank}. Score: {score:.4f}")
        print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
        print(f"   Preview: {chunk['text'][:150]}...")

print("\n" + "=" * 80)
print("[비교 결과]")
print("=" * 80)
print(f"일반 검색 Top-1 점수: {scores_orig[0]:.4f}")
print(f"CCH 검색 Top-1 점수: {scores_cch[0]:.4f}")
print(f"\n개선도: {(scores_cch[0] - scores_orig[0]):.4f}")

print("\n💡 CCH를 적용하면 청크가 더 명확한 컨텍스트를 포함하게 되어")
print("   암묵적 참조나 대명사가 많은 쿼리에서 검색 정확도가 향상됩니다.")


---
## 7️⃣ 추가 테스트: 다양한 쿼리로 CCH 효과 확인

이 셀에서는 여러 쿼리로 CCH의 효과를 추가 확인합니다.

**주요 내용:**
- 다양한 유형의 쿼리 테스트
- 일반 검색 vs CCH 검색 비교

**실행 결과:**
- 각 쿼리별 검색 결과와 점수 개선도가 출력됩니다.


In [ ]:
# 추가 테스트 쿼리
test_queries = [
    "What is the role of the scheduler in an operating system?",
    "How does paging work in memory management?",
    "Explain the concept of process synchronization"
]

print("=" * 80)
print("[추가 쿼리 테스트]")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"쿼리 {i}: {query}")
    print(f"{'='*80}")
    
    # 일반 검색
    indices_orig, scores_orig = search_with_index(query, INDEX_ORIGINAL, k=1)
    # CCH 검색
    indices_cch, scores_cch = search_with_index(query, index_cch, k=1)
    
    print(f"\n일반 검색 Top-1:")
    print(f"  Score: {scores_orig[0]:.4f}")
    print(f"  Chapter: {CHUNKS[indices_orig[0]].get('chapter_title', 'N/A')}")
    
    print(f"\nCCH 검색 Top-1:")
    print(f"  Score: {scores_cch[0]:.4f}")
    print(f"  Chapter: {CHUNKS[indices_cch[0]].get('chapter_title', 'N/A')}")
    
    improvement = scores_cch[0] - scores_orig[0]
    print(f"\n개선도: {improvement:+.4f} ({'↑' if improvement > 0 else '↓' if improvement < 0 else '→'})")

print("\n" + "=" * 80)
print("✅ CCH 테스트 완료!")
print("=" * 80)


---
## 8️⃣ 요약 및 다음 단계

### 학습 내용 요약

1. **CCH 개념**: 청크에 문서 컨텍스트(챕터 제목 등)를 헤더로 추가
2. **효과**: 임베딩이 더 풍부한 의미를 담게 되어 검색 정확도 향상
3. **구현**: 모든 청크에 헤더를 추가하고 새로운 FAISS 인덱스 생성
4. **검증**: 동일 쿼리에 대해 일반 검색보다 높은 유사도 점수 확인

### CCH의 장점

- ✅ 암묵적 참조와 대명사 문제 해결
- ✅ 청크 단독으로는 불분명한 컨텍스트 명시화
- ✅ 검색 정확도 향상
- ✅ 구현이 간단하고 추가 비용 최소화

### 다음 단계

- **Chapter 7.2**: HyDE (쿼리 확장 기법)
- **Chapter 7.3**: Reranking (검색 결과 재정렬)
- **Chapter 7.4**: Self-RAG (자기 평가 기반 검색)
- **Chapter 7.5**: 모든 기법 통합

### 생성된 파일

- CCH 적용 FAISS 인덱스: `/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch.index`
- 인덱스 메타데이터: `/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch_metadata.json`
